## The `threading` module

In [2]:
import time

def countdown(n):
    for i in range(n):
        print(n - i - 1, "left")
        time.sleep(1)

In [6]:
from threading import Thread

In [7]:
t = Thread(target=countdown, args=(3,))

In [8]:
t.start()

2 left
1 left
0 left


The second option:

In [9]:
class CountdownThread(Thread):
    def __init__(self, n):
        super().__init__()
        self.n = n
        
    def run(self):
        for i in range(self.n):
            print(self.n - i - 1, "left")
            time.sleep(1)

In [10]:
t = CountdownThread(3)

In [11]:
t

<CountdownThread(Thread-5, initial)>

In [13]:
t.start()

2 left
1 left
0 left


In [14]:
t.ident

123145422323712

In [15]:
t.name

'Thread-5'

In [16]:
import threading

In [17]:
threading.enumerate()

[<_MainThread(MainThread, started 4663078336)>,
 <Thread(Thread-2, started daemon 123145400229888)>,
 <Heartbeat(Thread-3, started daemon 123145405485056)>,
 <HistorySavingThread(IPythonHistorySavingThread, started 123145411813376)>,
 <ParentPollerUnix(Thread-1, started daemon 123145417068544)>]

### Joining threads

In [20]:
t = Thread(target=time.sleep, args=(5,))
t.start()
t.join()  # block for 5 seconds
t.join()

In [21]:
t.is_alive()  # after 5 seconds

False

### Daemons

In [22]:
t = Thread(target=time.sleep, args=(5,), daemon=True)

In [23]:
t.start()

In [24]:
t.daemon

True

### Terminating threads

Python doesn't have built-in mechanisms for terminating threads. Developers usually define a flag to determine whether thread running or not.

In [26]:
class Task:
    def __init__(self):
        self._running = True
        
    def terminate(self):
        self._running = False
        
    def run(self, n):
        while self._running:
            # do something
            pass

## Mutexes and Semaphores

The `threading` module includes:
- `Lock` – usual mutex
- `RLock` – recursive mutex*
- `Semaphore`
- `BoundedSemaphore`

[Python Thread Synchronization Primitives : Not Entirely What You Think](http://dabeaz.blogspot.com/2009/09/python-thread-synchronization.html)

### Thread-safe and slow counter

In [30]:
from threading import Lock

In [31]:
class SharedCounter:
    def __init__(self, value):
        self._value = value
        self._lock = Lock()
        
    def increment(self, delta=1):
        self._lock.acquire()
        self._value += delta
        self._lock.release()
        
    def get(self):
        return self._value

### Mutex doesn't have an owner

In [32]:
done = Lock()
def idle_release():
    print("Running!")
    time.sleep(15)
    done.release()

In [33]:
done.acquire()

True

In [35]:
Thread(target=idle_release).start()

Running!


In [36]:
done.acquire() and print("WAT?")  # after 16 secs

WAT?


## `Condition` objects

In [39]:
from threading import Condition
from collections import deque

In [40]:
q = deque()
is_empty = Condition()

In [41]:
def producer():
    while True:
        is_empty.acquire()
        q.append("ping")
        is_empty.notify()
        is_empty.release()
        
        
def consumer():
    # we don't want to spend the CPU time while waiting for a message
    while True:
        is_empty.acquire()
        while not q:
            # block the calling thread and wait for notify or notify_all
            is_empty.wait()
            q.popleft()
        is_empty.release()

⚠️ [Spurious wakeup - Wikipedia](https://en.wikipedia.org/wiki/Spurious_wakeup)

## The `queue` module

- `Queue` – FIFO queue
- `LifoQueue` – LIFO queue _aka_ stack
- `PriorityQueue` – queue of `(priority, item)` elements

## The `future` module

In [44]:
from concurrent.futures import Executor, ThreadPoolExecutor

In [45]:
executor = ThreadPoolExecutor(max_workers=4)
executor.submit(print, "Hello, world!")

Hello, world!


<Future at 0x1094a3810 state=finished returned NoneType>

In [46]:
list(executor.map(print, ["Knock?", "Knock!"]))

Knock?
Knock!


[None, None]

In [47]:
executor.shutdown()

We can use executors through context managers:

In [48]:
with ThreadPoolExecutor(max_workers=4) as executor:
    f = executor.submit(sorted, [4, 3, 1, 2])

In [50]:
f.running(), f.done(), f.cancelled()

(False, True, False)

In [51]:
f.result()

[1, 2, 3, 4]

In [52]:
f.exception()

In [53]:
f.add_done_callback(print)

<Future at 0x1095249d0 state=finished returned list>


## The `multiprocessing` module

In [59]:
import multiprocessing as mp

In [60]:
p = mp.Process(target=countdown, args=(5,))
p.start()

4 left
3 left
2 left
1 left
0 left


In [61]:
p.name, p.pid

('Process-1', 11253)

In [62]:
p.daemon

False

In [63]:
p.exitcode

0

In [64]:
def ponger(conn):
    conn.send("pong")

In [65]:
parent_conn, child_conn = mp.Pipe()

In [66]:
p = mp.Process(target=ponger, args=(child_conn, ))

In [67]:
p.start()

In [68]:
parent_conn.recv()  # there is a socket under the hood!

'pong'

In [69]:
p.join()